In [1]:
import os
import math
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn import svm, preprocessing

In [2]:
df = pd.read_csv('EURUSD1.csv', names=['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume'])
train = np.array(df.Close)

In [3]:
def CalcReturns(data):
    R = []    
    for i in range(len(data)-1):
        v = math.log(math.fabs(data[i+1]/data[i]))
        R.append(v)
        
    return R

def ZigZag(data, minsize):
    
    N = 0
    Z = {'zigzag': [], 'time': [], 'label' : []}
    
    T = N
    
    Count = 0;
    
    Max  = data[0]
    Min  = data[0]
    
    Flag = False
    
    PriceLow = 0
    PriceHigh = 0    
        
    while N < len(data):
        
        PriceLow = data[N]
        PriceHigh = data[N]        
        
        if Flag:
            
            if PriceHigh > Max:
                
                T = N
                Max = PriceHigh                
                
            elif (Max - PriceLow >= minsize):
                
                Z['time'].append(T)
                Z['label'].append(-1)
                Z['zigzag'].append(Max)
                
                Flag = False
                Count = Count + 1                                
                
                T = N
                Min = PriceLow                
                
        else:
               
            if PriceLow < Min:
                
                T = N
                Min = PriceLow                
                    
            elif (PriceHigh - Min >= minsize):
                    
                Z['time'].append(T)
                Z['label'].append(1)
                Z['zigzag'].append(Min)
                
                Flag = True
                Count = Count + 1                
                
                T = N
                Max = PriceHigh                 
    
        N = N + 1    

    return Z

def BuildData3(zigzag, returns, lag):
    
    D = []
    L = []
    
    D0 = []
    D1 = []
    
    L0 = []
    L1 = []
    
    N = len(returns) + 1
    Count = len(Z['time'])

    for i in range(0,N-lag):        
        try:
            index = zigzag["time"].index(i+lag-1)
            
            D0.append(returns[i:i+lag-1])
            
            if zigzag["label"][index] == 1:                
                L0.append([0, 0, 1])
            else:
                L0.append([1, 0, 0])
            
        except:
            
            D1.append(returns[i:i+lag-1])
            L1.append([0, 1, 0])

    Count0 = len(D0)
    Count1 = len(D1)
    
    #print(Count0, Count1)
    
    N = Count0 / 2
    if Count1 > N:        
        D1 = np.random.permutation(D1)
        
        D = D1[:N]
        L = L1[:N]
        
    else:        
        D = D1
        L = L1
    
    D_temp = np.concatenate((D, D0), axis=0)
    L_temp = np.concatenate((L, L0), axis=0)
    
    I = range(len(D_temp))
    I = np.random.permutation(I)
    
    D = []
    L = []
    
    for i in I:
        D.append(D_temp[i])
        L.append(L_temp[i])
    
    D = np.array(D)
    L = np.array(L)   
    
    D = preprocessing.scale(D)
    
    return D, L

In [4]:
Z = ZigZag(train, 0.0004)
returns = CalcReturns(train)

D, L = BuildData3(Z, returns, 61)
t_count = 1024

dims = D.shape[1]
print(D.shape)

(7609, 60)


In [5]:
def neural_network_model(x):

    hl1 = {'w' : tf.Variable(tf.random_normal([dims, n_nodes_hl1])),
           'b' : tf.Variable(tf.random_normal([n_nodes_hl1]))}

    hl2 = {'w' : tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
           'b' : tf.Variable(tf.random_normal([n_nodes_hl2]))}

    hl3 = {'w' : tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
           'b' : tf.Variable(tf.random_normal([n_nodes_hl3]))}
    
    hl4 = {'w' : tf.Variable(tf.random_normal([n_nodes_hl3, n_nodes_hl4])),
           'b' : tf.Variable(tf.random_normal([n_nodes_hl4]))}

    out = {'w' : tf.Variable(tf.random_normal([n_nodes_hl4, n_classes])),
           'b' : tf.Variable(tf.random_normal([n_classes]))}

    l1 = tf.matmul(x, hl1['w']) + hl1['b']
    l1 = tf.nn.relu(l1)

    l2 = tf.matmul(l1, hl2['w']) + hl2['b']
    l2 = tf.nn.relu(l2)

    l3 = tf.matmul(l2, hl3['w']) + hl3['b']
    l3 = tf.nn.relu(l3)
    
    l4 = tf.matmul(l3, hl4['w']) + hl4['b']
    l4 = tf.tanh(l4)

    y = tf.matmul(l1, out['w']) + out['b']
    return y

In [6]:
batch_size = 100

def train_neural_network(x,y):
    nn = neural_network_model(x)
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = nn, labels = y))
    optimizer = tf.train.AdamOptimizer(0.6).minimize(cost)   
    
    correct = tf.equal(tf.argmax(nn,1), tf.argmax(y_train, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
    
    saver = tf.train.Saver()
    
    with tf.Session() as session:
        session.run(tf.initialize_all_variables())

        epoch_loss = 0
        for epoch in range(hm_epochs):
            epoch_loss = 0
            for i in range(int((len(D) - t_count)/batch_size)):
                
                epoch_x = D[i*batch_size : (i+1)*batch_size]
                epoch_y = L[i*batch_size : (i+1)*batch_size]
                
                _, c = session.run([optimizer, cost], feed_dict = {x: epoch_x, y: epoch_y})
                epoch_loss += c
                
            if epoch % 100 == 0:
                print('Epoch', epoch, 'completed out of', hm_epochs, 'loss', epoch_loss)
                saver.save(session, 'data/fxnn.ckpt')

        print('Last epoch loss: ', epoch_loss)
        return nn, correct, accuracy
        

In [7]:
def test_neural_network(x_test, y_test, nn, correct, accuracy):
    saver = tf.train.Saver()
    with tf.Session() as session:
        saver.restore(session, 'data/fxnn.ckpt')
        r = tf.cast(correct, 'float').eval({x_train:x_test, y_train:y_test})
        
        test_buy  = 0.0
        test_sell = 0.0
        test_hold = 0.0

        correct_buy  = 0.0
        correct_sell = 0.0
        correct_hold = 0.0

        incorrect_buy  = 0.0
        incorrect_sell = 0.0
        incorrect_hold = 0.0
        
        for i in range(len(r)):
            if L[i][2] == 1:
                test_buy += 1.0;
            elif L[i][1] == 1:
                test_hold += 1.0;
            elif L[i][0] == 1:
                test_sell += 1.0;
                
            if r[i] == 1:
                if L[i][2] == 1:
                    correct_buy += 1.0;
                elif L[i][1] == 1:
                    correct_hold += 1.0;
                elif L[i][0] == 1:
                    correct_sell += 1.0;
            else:
                if L[i][2] == 1:
                    incorrect_buy += 1.0;
                elif L[i][1] == 1:
                    incorrect_hold += 1.0;
                elif L[i][0] == 1:
                    incorrect_sell += 1.0;
        
        print('Accuracy:',accuracy.eval({x_train:x_test, y_train:y_test}))
        
        print( "Test buy:  ", test_buy  )
        print( "Test sell: ", test_sell )
        print( "Test hold: ", test_hold )

        print( "Correct buy:  ", correct_buy,  "Incorrect buy:  ", incorrect_buy,  "Accuracy: ", (correct_buy/test_buy)*100 )
        print( "Correct sell: ", correct_sell, "Incorrect sell: ", incorrect_sell, "Accuracy: ", (correct_sell/test_sell)*100 )
        print( "Correct hold: ", correct_hold, "Incorrect hold: ", incorrect_hold, "Accuracy: ", (correct_hold/test_hold)*100 )
        
            

In [8]:
n_nodes_hl1 = 1000
n_nodes_hl2 = 1000
n_nodes_hl3 = 1000
n_nodes_hl4 = 1000

n_classes = 3
hm_epochs = 1000

x_train = tf.placeholder('float', [None, 60])
y_train = tf.placeholder('float')

nn, correct, accuracy = train_neural_network(x_train, y_train)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
('Epoch', 0, 'completed out of', 1000, 'loss', 32264.30387878418)
('Epoch', 100, 'completed out of', 1000, 'loss', 256.00764094293118)
('Epoch', 200, 'completed out of', 1000, 'loss', 182.26772302761674)
('Epoch', 300, 'completed out of', 1000, 'loss', 230.61531361669768)
('Epoch', 400, 'completed out of', 1000, 'loss', 8858.4657707214355)
('Epoch', 500, 'completed out of', 1000, 'loss', 378.63081178069115)
('Epoch', 600, 'completed out of', 1000, 'loss', 134.10962216556072)
('Epoch', 700, 'completed out of', 1000, 'loss', 11864.234266143292)
('Epoch', 800, 'completed out of', 1000, 'loss', 16304.535191189498)
('Epoch', 900, 'completed out of', 1000, 'loss', 119.80659198644571)
('Last epoch loss: ', 2865.2229010617011)


In [9]:
x_test = D[len(D)-t_count:]
y_test = L[len(L)-t_count:]

test_neural_network(x_test, y_test, nn, correct, accuracy)

('Accuracy:', 0.66503906)
('Test buy:  ', 330.0)
('Test sell: ', 341.0)
('Test hold: ', 353.0)
('Correct buy:  ', 232.0, 'Incorrect buy:  ', 98.0, 'Accuracy: ', 70.3030303030303)
('Correct sell: ', 228.0, 'Incorrect sell: ', 113.0, 'Accuracy: ', 66.86217008797654)
('Correct hold: ', 221.0, 'Incorrect hold: ', 132.0, 'Accuracy: ', 62.606232294617556)


In [16]:
from tensorflow.contrib.tensorboard.plugins import projector

LOG_DIR = 'data/logs'
metadata_path = os.path.join(LOG_DIR, 'metadata.tsv')

data = tf.Variable(D, name='fxnn')

with open(metadata_path, 'w') as metadata_file:
    for label in L:
        if label[0] == 1:
            metadata_file.write('-1\n')
        elif label[1] == 1:
            metadata_file.write('0\n')
        elif label[2] == 1:
            metadata_file.write('1\n')

with tf.Session() as sess:
    saver = tf.train.Saver([data])

    sess.run(data.initializer)
    saver.save(sess, os.path.join(LOG_DIR, 'fxnn.ckpt'))
    
    config = projector.ProjectorConfig()
    
    # One can add multiple embeddings.
    embedding = config.embeddings.add()
    embedding.tensor_name = data.name
    
    # Link this tensor to its metadata file (e.g. labels).
    embedding.metadata_path = 'logs/metadata.tsv'
    
    # Saves a config file that TensorBoard will read during startup.
    projector.visualize_embeddings(tf.summary.FileWriter(LOG_DIR), config)